## Application #1: nbconvert


### Additional resource:

Andreas Mueller: [*Writing (and publishing) a book written in Jupyter notebooks*](https://conferences.oreilly.com/jupyter/jup-ny/public/schedule/detail/60087)

## Distinction between use cases for nbformat and nbconvert

`nbformat`: creating and validating notebooks 

`nbconvert`: manipulate existing notebooks.


## Nbconvert use cases

* converting notebooks into other formats
    * web-display: html, slides (with reveal.js)
    * publishable documents: LaTeX/PDF, ASCIIDoc
    * plain-text: rst, markdown
    * executable scripts: e.g., `*.py` 

## Nbconvert use cases (cont.)

* manipulating notebook content 
    * cell magic (`%%R`) code highlighting
    * removing content
    * extracting images references for plain-text formats (LaTeX, markdown)
* executing notebooks from the command-line

## Command line Interface

* [NbConvertApp](#NbConvertApp)

Wrapper around around the base class functionality.

`NbConvertApp` defines the CLI `jupyter nbconvert`.

`NbConvertApp` is also what manages the configuration as established both on the command line and via traitlet config files.

### Nbconvert configuration: traitlets

Wide class of problems requires configurability.

Configuration is specified using traitlets in either
- dynamic configuration file: `jupyter_nbconvert_config.py`
- static configuration file: `jupyter_nbconvert_config.json`

## Base Classes

* [Exporters](#Exporters)
* [Preprocessors](#Preprocessors)
* [Templates (and filters)](#Templates-(and-filters)
* [Writers](#Writers-&-Postprocessors)
* [Postprocessors](#Writers-&-Postprocessors)

## Exporters

Orchestration layer.

Keyed to the `--to <exporter_name>` command line argument.

Exporters specify many aspects of conversion pipeline. E.g.:
* which preprocessors 
* whether & which template
* output format

### The resources dictionary

In addition the `NotebookNode` instance, exporters create a resources dictionary. 

This is useful for passing information that shouldn't exist in notebook itself.  
E.g.,:
* Notebook styling (for html export)
* pygments highlighting info (for LaTeX/PDF export)

## Preprocessors

Notebook to notebook transformations.

Can read and write to resources as necessary.

**ExecutePreprocessor**: This enables CLI execution.

**ClearOutputPreprocessor**: Removes outputs from notebook and output metadata.

**TagRemovePreprocessor**: Removes cells tagged with particular tags specified as traitlets.

## Templates (and filters)

Nbconvert uses Jinja2 templates. 

Templates inherit from one another.

Templates can access `filters` that can transform the content passed through them.

One of the most common filters passes the plaintext representation of a cell's source to `pandoc` for conversion.

### pandoc & pandocfilters

`pandoc` is a library that allows conversion between a wide variety of document formats.

It uses an internal json representation to represent the parsed text. 

`pandocfilters` is a python library that allows you to write filters that manipulate the intermediate json representation.

##  Writers & Postprocessors

**Writers** handles the Exporter's final output.

The main writer used by `nbconvert` is the `FilesWriter` which writes to disk.

**Postprocessors** manipulate the file after the Writer is finished.

The only postprocessor supported by `nbconvert` is the `ServePostProcessor`, which serves html page after it is written to disk.

## Entrypoints

In addition to being highly configurable, we have a mechanism for 3rd party libraries to register `Exporters` using entrypoints. 

1. Define `MyExporter` 
2. package it in `my_package`

use

```python
setup(…,
      entry_points = {'nbconvert.exporters': ['myexporter = my_package:MyExporter']}
)
```

3. `pip install my_package`
4. `jupyter nbconvert --to myexporter` uses your custom exporter.

## Multi-notebook workflows

Nbconvert is designed to handle workflows that work with individual notebooks. 

To work with multiple notebooks requires infrastructure inside of which you would use `nbconvert` or `nbformat`. 

For examples of these kinds of packages see:
- [bookbook](https://github.com/takluyver/bookbook) (html or pdf books from multiple notebooks)
- [multi_rise](https://github.com/mpacer/multi_rise) (one slideshow from many notebooks)